In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Results/RequiredDataSet.csv',header=None, names=['booking_date','channel','property_id','room_name','room_id','check_in','check_out','room_rate','booking_status'])
# data.head()


In [2]:
Results = pd.read_excel('Results/DataSetWithStayDate.xlsx')
# Results.head()

In [3]:
del Results['Old_Room_rate']
# Results.head()

In [29]:
scrapyData = pd.read_csv('Results/eventData2016.csv',header=None, names=['Day','Stay_date','Event','HolidayIn'])
scrapeResult = scrapyData.dropna()
# scrapeResult.head()

In [30]:
scrapeResult = scrapyData.dropna()
# scrapeResult.head()

In [31]:
scrapeResult['Stay_date'] =   '2016-' + scrapeResult['Stay_date'] 
# scrapeResult.head()

In [32]:
scrapeResult['Stay_date'] = pd.to_datetime(scrapeResult.Stay_date)
# scrapeResult.head()

In [33]:
del scrapeResult['HolidayIn']
del scrapeResult['Day']
# scrapeResult.head()

In [34]:
newResults = Results.filter(['Property_id','Stay_date','Booking_status','Room_rate'], axis=1)
# newResults.head()

In [35]:
newResults = newResults[(newResults.Booking_status == 'COMPLETED') | (newResults.Booking_status == 'CONFIRMED')]
# newResults.head()

In [36]:
newResults = newResults[(newResults.Stay_date >= '2016-01-01') & (newResults.Stay_date <= '2016-12-31')]
# newResults.head()

In [37]:
newResults['Stay_date'] = pd.to_datetime(newResults.Stay_date)
# newResults.head()

In [38]:
res = pd.merge(left=newResults, left_on='Stay_date',right=scrapeResult, right_on='Stay_date', how='left')

#res = pd.merge(newResults, scrapeResult, on='Stay_date', how='left')

In [39]:
res = res.sort_values(by = 'Stay_date', ascending=True)
res1 = res.drop_duplicates()
res1 = res1.reset_index()
del res1['index']
res1 = res1.fillna('-')
# res1.head()

In [40]:
events = res1[(res1.Event != '-') ]
# events.head()

In [41]:
events['totalRevenue'] = events.groupby(['Property_id','Stay_date'])['Room_rate'].transform('sum')
# events

In [42]:
events['room_count'] = events.groupby(['Property_id','Stay_date'])['Room_rate'].transform('count')
# events.head()

In [43]:
events['ADR'] = events.apply(lambda events: (events['totalRevenue'] / events['room_count'] ) , axis=1)
# events.head()

In [44]:
del events['Room_rate']
del events['Booking_status']
events = events.drop_duplicates()
# events

In [45]:
events_tr = events["totalRevenue"].mean()
events_tr

84231.78493294818

In [46]:
events_adr = events["ADR"].unique().mean()
events_adr

9676.2632532102743

In [22]:
nonevents = res1[(res1.Event == '-') ]
# nonevents.head()

In [23]:
nonevents['totalRevenue'] = nonevents.groupby(['Property_id','Stay_date'])['Room_rate'].transform('sum')
# nonevents.head()

In [24]:
nonevents['room_count'] = nonevents.groupby(['Property_id','Stay_date'])['Room_rate'].transform('count')
# nonevents.head()

In [25]:
nonevents['ADR'] = nonevents.apply(lambda nonevents: (nonevents['totalRevenue'] / nonevents['room_count'] ) , axis=1)
# nonevents.head()

In [26]:
del nonevents['Room_rate']
del nonevents['Booking_status']
nonevents = nonevents.drop_duplicates()
# events

In [27]:
nonevents_tr = nonevents["totalRevenue"].unique().mean()
nonevents_tr

96789.603445720786

In [28]:
nonevents_adr = nonevents["ADR"].unique().mean()
nonevents_adr

9888.6461195476513